In [1]:
import pandas as pd

# source - IHME GBD 2019
df_alcohol = pd.read_csv("alcohol_use_disorders.csv")

# source - The World Bank Global Financial Development
df_risk = pd.read_csv("stock_price_volatility_g7.csv")

### Data Preperation

In [2]:
df_risk

,Series Name,Country Name,Country Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
0,Stock price volatility,Japan,JPN,15.041940,24.822410,21.984690,22.128020,18.792220,16.969580,15.808860,...,19.78590,20.18368,22.282340,18.27540,24.89919,21.02885,13.436020,16.53054,19.30813,19.26422
1,Stock price volatility,United States,USA,14.158380,16.071580,12.995750,9.947115,9.037936,9.045654,9.915802,...,21.04165,13.91965,11.291030,12.56322,15.72065,11.00858,11.120380,15.48678,26.74141,24.99284
2,Stock price volatility,"Korea, Rep.",KOR,25.140400,22.097270,23.658940,21.817150,16.847140,33.367690,28.735270,...,10.45077,12.51602,13.207930,10.99904,10.71165,12.95550,13.892220,20.90015,25.47351,20.18115
3,Stock price volatility,Germany,DEU,22.187430,23.957390,16.967760,13.197430,13.974090,13.706650,11.739120,...,26.59126,18.52831,14.450420,18.34303,23.42877,16.79665,12.499040,14.94337,24.47580,24.37885
4,Stock price volatility,Italy,ITA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.01824,27.62747,20.869730,24.10843,29.67103,24.85292,15.678930,17.27391,27.83454,26.71594
5,Stock price volatility,Canada,CAN,9.093241,9.116957,8.125984,7.820632,9.683888,9.569884,8.235611,...,17.01751,12.34927,9.530421,11.48874,14.97211,10.78296,8.210422,9.63318,24.27321,22.86709
6,Stock price volatility,France,FRA,16.837150,21.870120,18.562440,18.661340,17.039800,17.742850,16.047440,...,27.35640,20.78452,15.406280,18.41038,23.62713,17.37942,11.669650,13.72222,24.65057,24.61805
7,Stock price volatility,United Kingdom,GBR,12.611740,13.541030,12.914880,13.199880,10.359250,10.528230,8.326374,...,19.47260,14.07119,11.244900,12.60134,17.39943,13.37846,9.617433,11.70249,21.54198,22.01797


In [3]:
df_risk = df_risk.drop(["Series Name", "Country Code"], axis = 1)
df_risk = df_risk.rename(columns={"Country Name": "Country"})
df_risk.loc[2, "Country"] = "South Korea"

In [4]:
#Rename year for df_risk

year_index = {}
year_list = []

for i_year in range(1990, 2022):
    year_list.append(i_year)

for year in year_list:
    target_index = f"{year}"+" [YR"+f"{year}"+"]"
    year_index[target_index] = year
    
df_risk = df_risk.rename(columns=year_index)
df_risk = df_risk.sort_values("Country")

In [5]:
alcohol_code = "Current number of cases of alcohol use disorders per 100 people, in both sexes aged age-standardized"
df_alcohol = df_alcohol.rename(columns={"Entity": "Country", alcohol_code: "Alcohol"})
df_alcohol = df_alcohol.drop(["Code"], axis = 1)

In [6]:
#Select Countries for df_alcohol
country_index = []
country_list = list(df_risk["Country"].unique())
for i_country in range(6840):
    if df_alcohol["Country"][i_country] not in country_list:
        country_index.append(i_country)
df_alcohol = df_alcohol.drop(country_index, axis=0)
df_alcohol = df_alcohol.sort_values("Country")

In [7]:
#Convert df_alcohol into time series form
df_alc = pd.DataFrame()
year_list = []

df_alc["Country"] = country_list

for i_year in range(1990, 2020):
    year_list.append(i_year)
    
for j_year in year_list:
    df_alc[str(j_year)] = list(df_alcohol.loc[df_alcohol["Year"]==j_year, "Alcohol"])
    
df_alcohol = df_alc

In [8]:
#transpose dataframes

index_list = [5, 6, 3, 4, 0, 2, 7, 1]
index = {}
count = 0

df_risk = df_risk.transpose()
for i_index_1 in index_list:
    index[i_index_1] = country_list[count]
    count += 1
    
df_risk = df_risk.rename(columns=index)
df_risk = df_risk.drop(["Country"], axis=0)

In [9]:
index = {}
count = 0

df_alcohol = df_alcohol.transpose()

for i_index_2 in range(8):
    index[i_index_2] = country_list[count]
    count += 1
    
df_alcohol = df_alcohol.rename(columns=index)
df_alcohol = df_alcohol.drop(["Country"], axis=0)

In [10]:
#Handling Missing Values

# df_alcohol.isna().any()
# df_risk.isna().sum()

#Fill Missing Values with Mean

df_risk = df_risk.fillna(df_risk["Italy"][9:].mean())

In [11]:
#Trim Risk Data to match Alcohol Data

df_risk = df_risk.drop([2020, 2021], axis=0)

In [16]:
#Adjust the index type of Alcohol DF

rename = {}
for name in range(1990, 2020):
    rename[str(name)] = name
    
df_alcohol = df_alcohol.rename(index = rename)

Canada            1.717368
France            2.135233
Germany           2.491382
Italy             1.521617
Japan              0.60494
South Korea       3.082899
United Kingdom    2.091263
United States     2.903162
Name: 1990, dtype: object

### Data Exploration

In [20]:
# Preparation for Correlation Analysis

def DataFrame_Maker(Name, df_1, df_2, col_1, col_2):
    material = {}
    material[col_1] = list(df_1[Name])
    material[col_2] = list(df_2[Name])
    return(pd.DataFrame(material))

# Make dictionary containing times series df for each countries
countries_df = {}
for country in country_list:
    countries_df[country] = DataFrame_Maker(country, df_risk, df_alcohol, "Risk", "Alcohol")

# Make dictionary containing horizontal df for each time period
years_df = {}    
for year in list(range(1990, 2020)):
    years_df[str(year)] = DataFrame_Maker(year, df_risk.transpose(), df_alcohol.transpose(), "Risk", "Alcohol")

In [38]:
# Correlation Analysis

corr_countrywise = {}
for country in country_list:
    data = []
    data.append(countries_df[country].corr()["Alcohol"]["Risk"])
    corr_countrywise[country] = data
df_corr_country = pd.DataFrame(corr_countrywise)

corr_yearwise = {}
for year in list(range(1990, 2020)):
    data = []
    data.append(years_df[str(year)].corr()["Alcohol"]["Risk"])
    corr_yearwise[year] = data
df_corr_year = pd.DataFrame(corr_yearwise)

In [42]:
df_corr_country.transpose().describe()

,0
count,8.000000
mean,-0.028082
std,0.224286
min,-0.346655
25%,-0.172750
50%,-0.030347
75%,0.157419
max,0.277337


In [43]:
df_corr_year.transpose().describe()

,0
count,30.000000
mean,-0.260206
std,0.342441
min,-0.732976
25%,-0.571370
50%,-0.324676
75%,-0.020083
max,0.423874
